In [16]:

command_str = '{sys.executable} -m pip install --upgrade shapely'.format(sys=sys)
print(command_str)
!{command_str}

C:\Users\577342\AppData\Local\Continuum\anaconda3\python.exe -m pip install --upgrade shapely


In [1]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

import choropleth_utils
import numpy as np
%run ../../load_magic/environment.py
%run ../../load_magic/storage.py
%pprint

c = choropleth_utils.ChoroplethUtilities()
notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
print(['c.{}'.format(fn) for fn in dir(c) if not fn.startswith('_')])
text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
dir()

Pretty printing has been turned OFF
D:\Documents\Repositories\notebooks\StatsByCountry\ipynb\Scrape Google Autocomplete by State.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']
['c.axes_str', 'c.clean_up_state_common_dict', 'c.clean_up_state_merge_dataframe', 'c.clean_up_state_unique_dict', 'c.clean_up_suggestion_list_dict', 'c.column_description_dict', 'c.conjunctify_list', 'c.copy_file_path', 'c.create_label_line_file', 'c.create_state_first_dict', 'c.create_country_colored_labeled_map', 'c.create_country_colored_map', 'c.create_us_google_suggest_labeled_map', 'c.create_country_labeled_map', 'c.figure_str', 'c.fill_style_prefix', 'c.fill_style_str', 'c.get_colorbar_xml', 'c.get_column_description', 'c.get_google_suggestion_list', 'c.get_legend_xml', 'c.get_style_list', 'c.get_tfidf_lists', '

['Config', 'In', 'Out', 'RandomForestClassifier', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'c', 'choropleth_utils', 'csv', 'exit', 'get_classifier', 'get_data_structs_df', 'get_datastructure_prediction', 'get_dir_tree', 'get_importances', 'get_input_sample', 'get_ipython', 'get_module_version', 'get_notebook_path', 'get_struct_name', 'ipykernel', 'json', 'jupyter_config_dir', 'notebook_path', 'notebookapp', 'np', 'os', 'pd', 'pickle', 'preprocess_data', 'quit', 're', 's', 'sys', 'text_editor_path', 'urllib']

In [34]:

import xml.etree.ElementTree as et

svg_dir = os.path.abspath(os.path.join(s.data_folder, 'svg'))

In [35]:

xml_str = et.tostring(root, encoding='unicode')
with open(file_path, 'w') as f:
    print(xml_str.strip(), file=f)

In [3]:

from svgpathtools import Line, Path
from shapely.geometry import Polygon

# Convert paths to polygons
def path_to_poly(inpath):
    points = []
    for path in inpath:
        if isinstance(path, Line):
            points.append([path.end.real, path.end.imag])
        else:
            num_segments = ceil(path.length() / minimum_stitch)
            for seg_i in range(int(num_segments + 1)):
                points.append([path.point(seg_i / num_segments).real,
                                path.point(seg_i / num_segments).imag])
    
    return Polygon(points)
print(['Path.{}'.format(fn) for fn in dir(Path) if not fn.startswith('_')])

['Path.T2t', 'Path.append', 'Path.area', 'Path.bbox', 'Path.clear', 'Path.closed', 'Path.continuous_subpaths', 'Path.count', 'Path.cropped', 'Path.curvature', 'Path.d', 'Path.derivative', 'Path.end', 'Path.extend', 'Path.ilength', 'Path.index', 'Path.insert', 'Path.intersect', 'Path.isclosed', 'Path.isclosedac', 'Path.iscontinuous', 'Path.joins_smoothly_with', 'Path.length', 'Path.normal', 'Path.point', 'Path.pop', 'Path.radialrange', 'Path.remove', 'Path.reverse', 'Path.reversed', 'Path.rotated', 'Path.scaled', 'Path.start', 'Path.t2T', 'Path.translated', 'Path.unit_tangent']


In [4]:

from svg.path import parse_path
import svgpathtools

In [28]:

us_stats_df = s.load_object('us_stats_df')
us_stats_df['centroid_x'] = np.nan
us_stats_df['centroid_y'] = np.nan
match_series = c.all_countries_df.outline_d.isnull()
for state_name, row_series in c.all_countries_df[~match_series].iterrows():
    path_obj = svgpathtools.parse_path(pathdef=row_series.outline_d, current_pos=0j)
    poly_obj = path_to_poly(path_obj)
    us_stats_df.loc[state_name, 'centroid_x'] = poly_obj.centroid.x
    us_stats_df.loc[state_name, 'centroid_y'] = poly_obj.centroid.y
us_stats_df.loc['Hawaii', 'centroid_x'] = 418.44421
us_stats_df.loc['Hawaii', 'centroid_y'] = 525.24959
s.store_objects(us_stats_df=us_stats_df)

In [13]:

us_stats_df = s.load_object('us_stats_df')
match_series = c.all_countries_df.label_line_d.isnull()
for state_name, row_series in c.all_countries_df[~match_series].iterrows():
    centroid_x = row_series.centroid_x
    centroid_y = row_series.centroid_y
    label_line_d = row_series.label_line_d
    d_list = label_line_d.split(' ')
    d_list[1] = '{},{}'.format(centroid_x, centroid_y)
    print('{}: from {} to {}'.format(state_name, label_line_d, ' '.join(d_list)))
    us_stats_df.loc[state_name, 'label_line_d'] = ' '.join(d_list)
s.store_objects(us_stats_df=us_stats_df)

Connecticut: from m 927.31957,154.64115 18.94647,32.60556 to m 926.7483133756758,155.86053827021178 18.94647,32.60556
Delaware: from m 903.52633,222.93657 64.68237,16.47902 to m 902.3632590085718,220.5912415446095 64.68237,16.47902
Hawaii: from m 418.44421,525.24959 -15.71816,33.03223 to m 418.44421,525.24959 -15.71816,33.03223
Maryland: from m 881.15683,220.02489 81.39157,51.3884 to m 894.2228756234892,223.57596220248865 81.39157,51.3884
New Hampshire: from m 931.54948,121.15437 58.69,-8.01921 to m 928.9497514056254,109.98702410025666 58.69,-8.01921
New Jersey: from m 907.2275,191.30037 44.94279,20.62081 to m 906.6006462142022,195.74062521568754 44.94279,20.62081
Rhode Island: from m 942.82924,148.82503 26.87755,13.92345 to m 942.9759028208904,148.34133440231676 26.87755,13.92345
Vermont: from M 911.37814,98.736311 884.28111,80.375484 to M 911.5622768467056,107.85171527956979 884.28111,80.375484
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\us_stats_df.pi

In [3]:

[cn for cn in c.all_countries_df.columns if not np.issubdtype(c.all_countries_df[cn].dtype, np.number)]

['district_abbreviation', 'State_Region', 'Google_Suggest_Unique', 'label_line_d', 'Google_Suggest_Common', 'Google_Suggest_First', 'outline_d']

In [5]:

numeric_column_name = 'text_x'
string_column_name = 'State_Region'
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                string_column_name=string_column_name,
                                                df=c.all_countries_df)
!"{text_editor_path}" "{os.path.abspath(svg_file_path)}"

In [11]:

[cn for cn in c.all_countries_df.columns if 'gun' in cn.lower()]

['Guns_Rank', 'Guns_Per_Capita', 'Guns_Registered', 'Total_Gun_Murder_Deaths_2010', 'Gun_Ownership_Percent_2013', 'Gun_Murder_Rate_2010', 'Gun_Suicide_Deaths', 'Gun_Suicide_Rate']

In [8]:

import numpy as np

string_column_name = None
columns_list = [cn for cn in c.all_countries_df.columns if np.issubdtype(c.all_countries_df[cn].dtype, np.number)]
for numeric_column_name in columns_list:
    svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                    string_column_name=string_column_name,
                                                    df=c.all_countries_df)

In [9]:

!start %windir%\explorer.exe "{os.path.abspath(os.path.join(svg_file_path, '../'))}"

In [5]:

numeric_column_name = 'Gun_Murder_Rate_2010'
string_column_name = 'Google_Suggest_Unique'
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                string_column_name=string_column_name,
                                                df=c.all_countries_df)
!"{text_editor_path}" "{os.path.abspath(svg_file_path)}"

In [14]:

string_column_name = 'Google_Suggest_Common'
svg_file_path = c.create_country_labeled_map(string_column_name=string_column_name,
                                        df=c.all_countries_df)
!"{text_editor_path}" "{os.path.abspath(svg_file_path)}"

In [15]:

string_column_name = 'Google_Suggest_First'
svg_file_path = c.create_country_labeled_map(string_column_name=string_column_name,
                                        df=c.all_countries_df)
!"{text_editor_path}" "{os.path.abspath(svg_file_path)}"

In [17]:

string_column_name = 'State_Region'
svg_file_path = c.create_country_labeled_map(string_column_name=string_column_name,
                                        df=c.all_countries_df)
!"{text_editor_path}" "{os.path.abspath(svg_file_path)}"

In [3]:

string_column_name = 'Google_Suggest_Unique'
svg_file_path = c.create_country_labeled_map(string_column_name=string_column_name,
                                        df=c.all_countries_df)
!"{text_editor_path}" "{os.path.abspath(svg_file_path)}"

In [3]:

c.create_label_line_file()

In [5]:

from xml.dom.minidom import parse, parseString

def print_xml(xml_obj):
    xml_str = et.tostring(xml_obj, encoding='unicode')
    reparsed = parseString(xml_str)
    xml_str = reparsed.toprettyxml()
    xml_str = re.sub(r'\n+[ \t]+\n+', r'\n', xml_str)
    xml_str = re.sub(r'\n+[ \t]+\n+', r'\n', xml_str)
    print(xml_str)

In [3]:

states_file_path = c.create_country_labeled_map(df=c.all_countries_df, column_name='Google_Suggest_Unique')
!"{text_editor_path}" "{os.path.abspath(states_file_path)}"

In [3]:

state_name = 'Alabama'
print(c.get_google_suggestion_list(state_name))

The whole list for Alabama is: good, republican, good at football, bad at kicking, bad this year, trashy, dangerous, humid, religious, and cheap.


In [4]:

#print(c.all_countries_df.columns.tolist())
columns_list = ['Google_Suggest_Common', 'Google_Suggest_Unique', 'Google_Suggest_First', 'text_x', 'text_y', 'label_line_d']
c.all_countries_df[columns_list].sample(10).T

,Georgia,Arizona,Florida,Nebraska,Mississippi,Colorado,Maine,Kansas,Montana,Hawaii
Google_Suggest_Common,cold,expensive,expensive,cold,boring,expensive,expensive,cold,expensive,expensive
Google_Suggest_Unique,popular for filming,sunny,windy,flooded,awful,healthy,creepy,windy,warm this winter,volcanically active
Google_Suggest_First,cheap,hot,hot,bad,poor,cold,safe,windy,big,expensive
text_x,812.8,326.1,829.005,524.76,697.751,432.76,953.983,541.695,391.831,401.689
text_y,386.424,353.337,431.703,231.702,422.315,280.009,67.3263,289.337,106.26,566.369
label_line_d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"m 418.44421,525.24959 -15.71816,33.03223"


In [6]:

c.create_label_line_file()
!"{text_editor_path}" "{os.path.abspath(c.label_line_file_path)}"

In [7]:

states_file_path = c.create_us_google_suggest_labeled_map(cu_str='unique')
!"{text_editor_path}" "{os.path.abspath(states_file_path)}"
states_file_path = c.create_us_google_suggest_labeled_map(cu_str='common')
!"{text_editor_path}" "{os.path.abspath(states_file_path)}"
states_file_path = c.create_us_google_suggest_labeled_map(cu_str='first')
!"{text_editor_path}" "{os.path.abspath(states_file_path)}"


---
# Do some ft-idf calculations

In [8]:

sorted_d, reverse_sorted_d = c.get_tfidf_lists()
print(sorted_d[:25])
print(reverse_sorted_d[:25])

[('expensive', 3.591617604543698), ('cold', 3.706027955721442), ('boring', 3.768548312702776), ('cheap', 3.8352396872014487), ('humid', 4.067041301258772), ('poor', 4.258096538021482), ('republican', 4.312163759291758), ('dangerous', 4.369322173131707), ('bad', 4.429946794948141), ('weird', 4.429946794948141), ('conservative', 4.494485316085712), ('hot', 4.494485316085712), ('popular', 4.494485316085712), ('windy', 4.494485316085712), ('important', 4.563478187572664), ('democratic', 4.717628867399922), ('flat', 4.717628867399922), ('live', 4.717628867399922), ('warm', 4.804640244389551), ('rich', 4.899950424193877), ('backwards', 5.005310939851703), ('cloudy', 5.256625368132609), ('depressing', 5.256625368132609), ('small', 5.256625368132609), ('big', 5.410776047959867)]
[('active', 6.5093883366279774), ('alcohol', 6.5093883366279774), ('awful', 6.5093883366279774), ('basketball', 6.5093883366279774), ('beautiful', 6.5093883366279774), ('broke', 6.5093883366279774), ('business', 6.5093

In [9]:

c.state_common_dict

{'California': 'expensive', 'Colorado': 'expensive', 'Connecticut': 'expensive', 'Florida': 'expensive', 'Hawaii': 'expensive', 'Idaho': 'expensive', 'Illinois': 'expensive', 'Maine': 'expensive', 'Maryland': 'expensive to live in', 'Massachusetts': 'expensive', 'Minnesota': 'expensive', 'Alaska': 'expensive', 'Arizona': 'expensive', 'Montana': 'expensive', 'New Hampshire': 'expensive', 'New Jersey': 'expensive', 'New York': 'expensive', 'North Dakota': 'expensive', 'Oregon': 'expensive', 'Rhode Island': 'expensive', 'Texas': 'expensive', 'Vermont': 'expensive', 'Virginia': 'expensive', 'Washington': 'expensive', 'Wisconsin': 'expensive', 'Wyoming': 'expensive', 'Georgia': 'cold', 'Iowa': 'cold', 'Kansas': 'cold', 'Michigan': 'cold', 'Missouri': 'cold', 'Nebraska': 'cold', 'New Mexico': 'cold', 'North Carolina': 'cold', 'Oklahoma': 'cold', 'Pennsylvania': 'cold', 'South Dakota': 'cold', 'Utah': 'cold', 'Delaware': 'boring', 'Kentucky': 'boring', 'Mississippi': 'boring', 'Arkansas': 'bo

In [10]:

c.state_first_dict

{'California': 'expensive', 'Colorado': 'cold', 'Connecticut': 'rich', 'Delaware': 'small', 'Florida': 'hot', 'Georgia': 'cheap', 'Hawaii': 'expensive', 'Kentucky': 'poor', 'Louisiana': 'poor', 'Idaho': 'boring', 'Illinois': 'broke', 'Indiana': 'conservative', 'Iowa': 'important', 'Kansas': 'windy', 'Maine': 'safe', 'Maryland': 'rich', 'Massachusetts': 'rich', 'Michigan': 'cold', 'Minnesota': 'cold', 'Mississippi': 'poor', 'Missouri': 'dangerous', 'Alabama': 'good', 'Alaska': 'cold', 'Arizona': 'hot', 'Arkansas': 'poor', 'Montana': 'big', 'Nebraska': 'bad', 'Nevada': 'empty', 'New Hampshire': 'rich', 'New Jersey': 'expensive', 'New Mexico': 'poor', 'New York': 'expensive', 'North Carolina': 'cheap', 'North Dakota': 'cold', 'Ohio': 'cheap', 'Oklahoma': 'cheap', 'Oregon': 'weird', 'Pennsylvania': 'conservative', 'Rhode Island': 'small', 'South Carolina': 'poor', 'South Dakota': 'cold', 'Tennessee': 'cheap', 'Texas': 'cheap', 'Utah': 'weird', 'Vermont': 'blue', 'Virginia': 'rich', 'Washin


---

In [11]:

len(c.suggestion_list_dict.keys())

51

In [12]:

print(set(c.all_countries_df.index) - set(c.suggestion_list_dict.keys()))

set()


In [13]:

suggestion_list_dict = c.scrape_suggestion_list_dictionary(False)

Getting the FireFox driver
Getting URL: https://www.google.com


In [14]:

suggestion_list_dict

{'California': ['expensive', 'popular', 'dry', 'hot', 'populated', 'bad', 'big', 'polluted', 'democratic', 'expensive reddit'], 'Colorado': ['cold', 'expensive', 'healthy', 'dry', 'high', 'popular', 'square', 'windy', 'expensive reddit', 'sunny'], 'Connecticut': ['rich', 'expensive', 'weird', 'small', 'strange', 'boring', 'dangerous', 'expensive to live in'], 'Delaware': ['small', 'cheap', 'flat', 'windy', 'boring', 'business friendly', 'popular for corporations', 'democratic', 'irrelevant', 'liberal'], 'Florida': ['hot', 'cheap', 'humid', 'flat', 'popular', 'populated', 'expensive', 'ghetto', 'rainy', 'windy'], 'Georgia': ['cheap', 'cold', 'poor', 'bad', 'hot', 'cheap to live in', 'humid', 'boring', 'popular for filming', 'backwards'], 'Hawaii': ['expensive', 'popular', 'hot', 'beautiful', 'expensive to live', 'warm', 'democratic', 'volcanically active', 'far away', 'famous'], 'Kentucky': ['poor', 'backwards', 'conservative', 'good at basketball', 'hilly', 'republican', 'humid', 'bori

In [ ]:

s.store_objects(suggestion_list_dict=suggestion_list_dict)

In [15]:

sum(list(c.suggestion_list_dict.values()), [])

['expensive', 'popular', 'dry', 'hot', 'populated', 'bad', 'big', 'polluted', 'democratic', 'expensive reddit', 'cold', 'expensive', 'healthy', 'dry', 'high', 'popular', 'square', 'windy', 'expensive reddit', 'sunny', 'rich', 'expensive', 'weird', 'small', 'strange', 'boring', 'dangerous', 'expensive to live in', 'small', 'cheap', 'flat', 'windy', 'boring', 'business friendly', 'popular for corporations', 'democratic', 'irrelevant', 'liberal', 'hot', 'cheap', 'humid', 'flat', 'popular', 'populated', 'expensive', 'ghetto', 'rainy', 'windy', 'cheap', 'cold', 'poor', 'bad', 'hot', 'cheap to live in', 'humid', 'boring', 'popular for filming', 'backwards', 'expensive', 'popular', 'hot', 'beautiful', 'expensive to live', 'warm', 'democratic', 'volcanically active', 'far away', 'famous', 'poor', 'backwards', 'conservative', 'good at basketball', 'hilly', 'republican', 'humid', 'boring', 'cheap', 'hot', 'poor', 'dangerous', 'french', 'humid', 'weird', 'hot', 'swampy', 'conservative', 'dirty', 

In [4]:

us_stats_df = s.load_object('us_stats_df')
us_stats_df.loc['Hawaii', 'text_x'] = 400.1615
us_stats_df.loc['Hawaii', 'text_y'] = 570.25653
us_stats_df.loc['Alabama', 'text_x'] = 748.14435
us_stats_df.loc['Alabama', 'text_y'] = 392.44476
us_stats_df.loc['Alaska', 'text_x'] = 220.50682
us_stats_df.loc['Alaska', 'text_y'] = 492.66138
us_stats_df.loc['Arizona', 'text_x'] = 326.09976
us_stats_df.loc['Arizona', 'text_y'] = 353.33734
us_stats_df.loc['Arkansas', 'text_x'] = 649.2536
us_stats_df.loc['Arkansas', 'text_y'] = 342.48181
us_stats_df.loc['California', 'text_x'] = 204.72113
us_stats_df.loc['California', 'text_y'] = 287.07516
us_stats_df.loc['Colorado', 'text_x'] = 432.75961
us_stats_df.loc['Colorado', 'text_y'] = 280.00891
us_stats_df.loc['Connecticut', 'text_x'] = 977.01733
us_stats_df.loc['Connecticut', 'text_y'] = 191.64752
us_stats_df.loc['Connecticut', 'label_line_d'] = 'm 927.31957,154.64115 18.94647,32.60556'
us_stats_df.loc['Delaware', 'text_x'] = 992.09674
us_stats_df.loc['Delaware', 'text_y'] = 243.09442
us_stats_df.loc['Delaware', 'label_line_d'] = 'm 903.52633,222.93657 64.68237,16.47902'
us_stats_df.loc['Florida', 'text_x'] = 829.00507
us_stats_df.loc['Florida', 'text_y'] = 431.70282
us_stats_df.loc['Georgia', 'text_x'] = 812.79968
us_stats_df.loc['Georgia', 'text_y'] = 386.42352
us_stats_df.loc['Hawaii', 'text_x'] = 401.6886
us_stats_df.loc['Hawaii', 'text_y'] = 566.36896
us_stats_df.loc['Hawaii', 'label_line_d'] = 'm 418.44421,525.24959 -15.71816,33.03223'
us_stats_df.loc['Idaho', 'text_x'] = 314.79205
us_stats_df.loc['Idaho', 'text_y'] = 167.07047
us_stats_df.loc['Illinois', 'text_x'] = 687.95764
us_stats_df.loc['Illinois', 'text_y'] = 247.85957
us_stats_df.loc['Indiana', 'text_x'] = 736.74451
us_stats_df.loc['Indiana', 'text_y'] = 262.52542
us_stats_df.loc['Iowa', 'text_x'] = 620.46136
us_stats_df.loc['Iowa', 'text_y'] = 213.45132
us_stats_df.loc['Kansas', 'text_x'] = 541.69531
us_stats_df.loc['Kansas', 'text_y'] = 289.3371
us_stats_df.loc['Kentucky', 'text_x'] = 760.86127
us_stats_df.loc['Kentucky', 'text_y'] = 284.59695
us_stats_df.loc['Louisiana', 'text_x'] = 649.51294
us_stats_df.loc['Louisiana', 'text_y'] = 410.27689
us_stats_df.loc['Maine', 'text_x'] = 953.98334
us_stats_df.loc['Maine', 'text_y'] = 67.326279
us_stats_df.loc['Maryland', 'text_x'] = 985.55103
us_stats_df.loc['Maryland', 'text_y'] = 274.39795
us_stats_df.loc['Maryland', 'label_line_d'] = 'm 881.15683,220.02489 81.39157,51.3884'
us_stats_df.loc['Massachusetts', 'text_x'] = 937.32983
us_stats_df.loc['Massachusetts', 'text_y'] = 140.9111
us_stats_df.loc['Michigan', 'text_x'] = 745.27258
us_stats_df.loc['Michigan', 'text_y'] = 174.73953
us_stats_df.loc['Minnesota', 'text_x'] = 599.13074
us_stats_df.loc['Minnesota', 'text_y'] = 112.43513
us_stats_df.loc['Mississippi', 'text_x'] = 697.75061
us_stats_df.loc['Mississippi', 'text_y'] = 422.31543
us_stats_df.loc['Missouri', 'text_x'] = 639.56989
us_stats_df.loc['Missouri', 'text_y'] = 289.03799
us_stats_df.loc['Montana', 'text_x'] = 391.83105
us_stats_df.loc['Montana', 'text_y'] = 106.2601
us_stats_df.loc['Nebraska', 'text_x'] = 524.76038
us_stats_df.loc['Nebraska', 'text_y'] = 231.70229
us_stats_df.loc['Nevada', 'text_x'] = 257.36636
us_stats_df.loc['Nevada', 'text_y'] = 239.53661
us_stats_df.loc['New Hampshire', 'text_x'] = 999.25446
us_stats_df.loc['New Hampshire', 'text_y'] = 109.4236
us_stats_df.loc['New Hampshire', 'label_line_d'] = 'm 931.54948,121.15437 58.69,-8.01921'
us_stats_df.loc['New Jersey', 'text_x'] = 960.73419
us_stats_df.loc['New Jersey', 'text_y'] = 208.55847
us_stats_df.loc['New Jersey', 'label_line_d'] = 'm 907.2275,191.30037 44.94279,20.62081'
us_stats_df.loc['New Mexico', 'text_x'] = 418.89343
us_stats_df.loc['New Mexico', 'text_y'] = 371.03604
us_stats_df.loc['New York', 'text_x'] = 880.10455
us_stats_df.loc['New York', 'text_y'] = 154.17368
us_stats_df.loc['North Carolina', 'text_x'] = 866.19855
us_stats_df.loc['North Carolina', 'text_y'] = 301.41946
us_stats_df.loc['North Dakota', 'text_x'] = 514.11646
us_stats_df.loc['North Dakota', 'text_y'] = 111.24364
us_stats_df.loc['Ohio', 'text_x'] = 784.07135
us_stats_df.loc['Ohio', 'text_y'] = 224.19022
us_stats_df.loc['Oklahoma', 'text_x'] = 566.31805
us_stats_df.loc['Oklahoma', 'text_y'] = 347.56897
us_stats_df.loc['Oregon', 'text_x'] = 221.30357
us_stats_df.loc['Oregon', 'text_y'] = 139.91429
us_stats_df.loc['Pennsylvania', 'text_x'] = 856.09741
us_stats_df.loc['Pennsylvania', 'text_y'] = 189.2137
us_stats_df.loc['Rhode Island', 'text_x'] = 983.90399
us_stats_df.loc['Rhode Island', 'text_y'] = 159.57837
us_stats_df.loc['Rhode Island', 'label_line_d'] = 'm 942.82924,148.82503 26.87755,13.92345'
us_stats_df.loc['South Carolina', 'text_x'] = 843.32721
us_stats_df.loc['South Carolina', 'text_y'] = 345.96466
us_stats_df.loc['South Dakota', 'text_x'] = 517.34576
us_stats_df.loc['South Dakota', 'text_y'] = 167.97604
us_stats_df.loc['Tennessee', 'text_x'] = 744.53644
us_stats_df.loc['Tennessee', 'text_y'] = 328.19617
us_stats_df.loc['Texas', 'text_x'] = 540.02362
us_stats_df.loc['Texas', 'text_y'] = 433.99121
us_stats_df.loc['Utah', 'text_x'] = 340.63342
us_stats_df.loc['Utah', 'text_y'] = 256.71301
us_stats_df.loc['Vermont', 'text_x'] = 882.93085
us_stats_df.loc['Vermont', 'text_y'] = 77.379402
us_stats_df.loc['Vermont', 'label_line_d'] = 'M 911.37814,98.736311 884.28111,80.375484'
us_stats_df.loc['Virginia', 'text_x'] = 867.47754
us_stats_df.loc['Virginia', 'text_y'] = 257.43704
us_stats_df.loc['Washington', 'text_x'] = 245.27382
us_stats_df.loc['Washington', 'text_y'] = 78.791374
us_stats_df.loc['West Virginia', 'text_x'] = 830.54102
us_stats_df.loc['West Virginia', 'text_y'] = 242.49905
us_stats_df.loc['Wisconsin', 'text_x'] = 661.45966
us_stats_df.loc['Wisconsin', 'text_y'] = 153.29938
us_stats_df.loc['Wyoming', 'text_x'] = 408.97885
us_stats_df.loc['Wyoming', 'text_y'] = 191.26553
s.store_objects(us_stats_df=us_stats_df)

Pickling to C:\Users\dev\Documents\repositories\notebooks\StatsByCountry\saves\pickle\us_stats_df.pickle
